In [1]:
import cv2
import numpy as np
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot

#define eye points
idListRight = [ 257,359,463,467,414,253,255,256,258,259,260 ,286,252,254,257,253]
idListLeft  = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]
#create detector
detector = FaceMeshDetector(maxFaces=1)
#chose our camera 
cap = cv2.VideoCapture(0)
#define live plot parameter
plotY = LivePlot(640, 360, [20, 50], invert=False)
plotX = LivePlot(640, 360, [20, 50], invert=False)
color = (0,200,0)
leftRatioList = []
RightRatioList = []
counterLeft = 0
counterRight = 0
leftBlinkCounter  = 0
RightBlinkCounter = 0

while True :
    _,frame = cap.read()
    #start detect our face 
    img,faces = detector.findFaceMesh(frame,draw=False)
    #check if face is exist
    if faces :
        face = faces[0]
        for id in idListLeft :
            cv2.circle(frame,face[id],1,color,cv2.FILLED)
        for id in idListRight :
            cv2.circle(frame,face[id],1,color,cv2.FILLED)
        #start extract the points to calulate ration for detection
        # left eye # right eye
        leftUp    = face[159]
        leftDown  = face[23]
        leftRight = face[243]
        leftLeft  = face[130]
        rightUp   = face[257]
        rightDown = face[253]
        rightRight= face[359]
        rightLeft = face[463]
        #calculate distance between points
        leftVerticalLenth   ,_ = detector.findDistance(leftUp,leftDown)
        leftHorizintalLenth ,_ = detector.findDistance(leftLeft,leftRight)
        rightVerticalLenth  ,_ = detector.findDistance(rightUp,rightDown)
        rightHorizintalLenth,_ = detector.findDistance(rightLeft,rightRight)
        cv2.line(frame,leftRight,leftLeft,color,1)
        cv2.line(img, leftUp, leftDown,color,1)
        cv2.line(frame,rightRight,rightLeft,color,1)
        cv2.line(img, rightUp, rightDown,color,1)
        #caluclat the ration
        leftRatio = int((leftVerticalLenth/leftHorizintalLenth)*100)#35
        leftRatioList.append(leftRatio)
        rightRatio= int((rightVerticalLenth /rightHorizintalLenth)*100)#45
        RightRatioList.append(rightRatio)
        if len(leftRatioList ) > 3 :
            leftRatioList.pop(0)
        averageLeftRatio = sum(leftRatioList)/len(leftRatioList)
        if  len(RightRatioList) > 3:   
            RightRatioList.pop(0)
        averageRightRatio = sum(RightRatioList)/len(RightRatioList)
    #check for left eye counters
    if averageLeftRatio <35 and counterLeft == 0:
        leftBlinkCounter +=1
        counterLeft = 1
    if counterLeft !=0:
        counterLeft+=1
        if counterLeft>35:
            counterLeft =0
    #check for left eye counters
    if averageRightRatio <50 and counterRight == 0 :
        counterRight = 1
        RightBlinkCounter +=1
    if counterRight !=0 :
        counterRight +=1
        if counterRight > 50:
            counterRight = 0
        # start calculate the ration for Two Eyes
        imgPlotRight = plotY.update(averageRightRatio, color)
        imgPlotLeft  = plotX.update(averageLeftRatio, color)
        #frame = cv2.resize(frame, (400, 300))
        #cvzone.putTextRect(frame, f'LBlink Count: {leftBlinkCounter}', (50, 100),colorR=color)  
        #cvzone.putTextRect(frame, f'RBlink Count: {RightBlinkCounter}', (200, 100),colorR=color) 
        cv2.putText(frame, f'LBlink Count: {leftBlinkCounter}', (10, 20), cv2.FONT_HERSHEY_COMPLEX, .6, color, 1)  
        cv2.putText(frame, f'RBlink Count: {RightBlinkCounter}', (180, 20), cv2.FONT_HERSHEY_SIMPLEX, .6, color, 1)  
        #imgStack = cvzone.stackImages([frame, imgPlotLeft, imgPlotRight], 3, 1)
        imgStack = cvzone.stackImages([frame], 1, 1)
        cv2.imshow("Image", imgStack)
        
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 